# Yelp API Restaurant Calls By GPS Coordinates

For each coordinate listed in grid_coordinates2.csv, return first 1k restaurants found via yelp api and extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id

Write data to restaurant_data_coord.csv

In [16]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [36]:
# Create dataframe from csv file
coords_df = pd.read_csv("Resources/grid_coordinates6.csv", 
                        names=["Lat", "Lng"])
coords_df = coords_df.dropna(how='all')
coords_df

,Lat,Lng
0,37.363733,-122.294708
2,37.363733,-122.239130
4,37.363733,-122.183552
6,37.363733,-122.127974
8,37.363733,-122.072396
10,37.363733,-122.016818
12,37.363733,-121.961240
14,37.363733,-121.905662
16,37.363733,-121.850084
18,37.363733,-121.794506


In [37]:
# Function returns up to 1000 restaurant listings for input coordinates
def get_restaurants(lat, lng, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls, radius 8046 meters = 5 miles
        params = {"term":"restaurants", "latitude":lat, "longitude":lng, "radius":4829, 
                  "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        
                        # Replace both price & missing category data with empty string
                        if biz["categories"] == []:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", "", biz["id"]])
                        else:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", biz["categories"][0]["title"], 
                                                    biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/restaurant_data_coordsTEST.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()
    
    # Returns total count of restaurants in coordinates
    return response["total"]


In [38]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , api_key))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates complete")
        print(f"Getting results for next coordinates.. there are {rem} coordinates left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 304 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 303 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 302 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of m

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 284 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 283 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 282 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 281 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
---------------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 262 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 261 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 260 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 259 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
---------------------------------------

Now processing query set 8 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 237 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 236 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 235 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 234 coordinates left
-----------------------------------------
No

Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 207 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 206 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 205 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 204 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
--

...

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 141 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 140 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 139 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 138 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
---------------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 107 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 106 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 105 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 104 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
---------------------------------------

Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 73 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 72 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 71 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 70 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now pr

Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 41 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 40 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 39 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing

Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 9 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 8 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 7 coordinates left
----------------------

In [25]:
# Show coords_df with updated total # of restaurants
coords_df.head()

,Lat,Lng,Total # of Restaurants
0,38.842665,-123.072800,0
2,38.842665,-123.017222,13
4,38.842665,-122.961644,0
6,38.842665,-122.906066,0
8,38.842665,-122.850488,0


In [26]:
# Save restaurants counts to csv
coords_df.to_csv("Resources/total_count.csv")